建立模擬資料

```
這個 data 資料夾有一個 transaction.csv

幫我建立這個 csv 假資料，用 code 建立

acct_num |  txn_date | txn_time | drcr | txn_amt
bk134| 2023-10-26 | 09:00 | 1 | 3103850
bk478| 2023-10-25 | 03:11  | 2 | 4u042

acct_num: 表示帳戶號碼，幫我建立100個帳戶
txn_date : 表示交易日期，幫我建立交易日期在 2023-10-20 到 2023-10-30日之間
txn_time : 表示交易時間，一天24 小時內都可以
drcr : 表示轉入或是轉出 1 是轉入 , 2 是轉出
txn_amt : 這筆交易的金額，金額範圍幫我寫 1- 1000000

請根據上述條件，建立1000筆的交易資料
```

In [20]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta

# 设置随机数种子以确保可重复性
np.random.seed(0)
random.seed(0)

# 创建一个Faker实例以生成虚假的帐户号码
fake = Faker()

# 创建一个空的DataFrame来存储交易数据
data = pd.DataFrame(columns=['acct_num', 'txn_date', 'txn_time', 'drcr', 'txn_amt'])

# 生成100个帐户号码

account_numbers = [fake.unique.random_number(digits=6) for _ in range(100)]

# 生成1000条交易记录
for _ in range(1000):
    acct_num = random.choice(account_numbers)
    txn_date = (datetime(2023, 10, 20) + timedelta(days=random.randint(0, 10))).date()
    txn_time = fake.time(pattern='%H:%M')
    drcr = random.randint(1, 2)
    txn_amt = random.randint(1, 1000000)  

    data = pd.concat([data, pd.DataFrame({'acct_num': [acct_num], 'txn_date': [txn_date], 'txn_time': [txn_time],
                                          'drcr': [drcr], 'txn_amt': [txn_amt]})], ignore_index=True)


for _ in range(1000):
    acct_num = random.choice(account_numbers)
    txn_date = (datetime(2023, 9, 20) + timedelta(days=random.randint(0, 10))).date()
    txn_time = fake.time(pattern='%H:%M')
    drcr = random.randint(1, 2)
    txn_amt = random.randint(1, 1000000)  

    data = pd.concat([data, pd.DataFrame({'acct_num': [acct_num], 'txn_date': [txn_date], 'txn_time': [txn_time],
                                          'drcr': [drcr], 'txn_amt': [txn_amt]})], ignore_index=True)

# 将数据保存为CSV文件
data.to_csv('data/transaction.csv', index=False)


In [21]:
pd.read_csv('./data/transaction.csv')

,acct_num,txn_date,txn_time,drcr,txn_amt
0,835030,2023-10-26,00:47,1,271494
1,786841,2023-10-27,13:11,2,962839
2,801924,2023-10-27,05:01,2,611721
3,301273,2023-10-28,06:09,1,295529
4,555321,2023-10-21,12:48,2,953939
...,...,...,...,...,...
1995,266828,2023-09-29,13:47,2,224127
1996,191070,2023-09-30,10:16,1,739999
1997,301273,2023-09-25,17:00,1,322447
1998,301273,2023-09-20,06:02,2,930700


In [23]:
# import sqlite3
# import pandas as pd


# conn = sqlite3.connect('database.db')

# # 创建一个数据库表格并将数据写入
# # data = pd.read_csv('data.csv')

# # 将数据写入数据库中的表格（如果表格不存在则会创建）
# data.to_sql('transactions', conn, if_exists='replace', index=False)

# # 关闭数据库连接
# conn.close()


特徵工程:計算每筆賬號，這段時間內，平均的交易金額

In [15]:
import pandas as pd

def calculate_average_txn_amount(data, start_date, end_date):
    """
    计算每个帐户在指定时间段内的平均交易金额。

    参数：
    data (DataFrame): 包含交易数据的DataFrame。
    start_date (str): 时间段的起始日期（格式：'YYYY-MM-DD'）。
    end_date (str): 时间段的结束日期（格式：'YYYY-MM-DD'）。

    返回：
    DataFrame: 包含每个帐户的平均交易金额的DataFrame。
    """
    # 将日期列转换为datetime类型
    data['txn_date'] = pd.to_datetime(data['txn_date'])

    # 筛选在指定时间段内的交易数据
    filtered_data = data[(data['txn_date'] >= start_date) & (data['txn_date'] <= end_date)]

    # 按帐户号分组并计算平均交易金额
    average_txn_amount = filtered_data.groupby('acct_num')['txn_amt'].mean().reset_index()

    return average_txn_amount


In [16]:
start_date, end_date = '2023-10-20' , '2023-10-30'

calculate_average_txn_amount(data, start_date, end_date)

,acct_num,txn_amt
0,16540,666265.0
1,23144,462648.2
2,28890,565576.1875
3,34759,467453.636364
4,41300,403012.631579
...,...,...
95,952179,529833.5
96,956657,535726.0
97,973183,352274.625
98,985985,501803.909091


In [24]:
import multiprocessing
import sqlite3
import pandas as pd

# 创建一个连接到数据库的函数
def create_db_connection():
    return sqlite3.connect('mydatabase.db')

# 定义一个函数来执行特征工程操作
def calculate_average_txn_amount(data, start_date, end_date):
    # 将日期列转换为datetime类型
    data['txn_date'] = pd.to_datetime(data['txn_date'])

    # 筛选在指定时间段内的交易数据
    filtered_data = data[(data['txn_date'] >= start_date) & (data['txn_date'] <= end_date)]

    # 按帐户号分组并计算平均交易金额
    average_txn_amount = filtered_data.groupby('acct_num')['txn_amt'].mean().reset_index()

    return average_txn_amount

# 定义一个函数来处理特定的acct_num的特征工程任务
def process_acct_num(acct_num):
    # 在这里执行特征工程操作，生成特征数据
    feature_data = calculate_average_txn_amount(data, start_date, end_date)
    
    # 连接到数据库
    conn = create_db_connection()
    
    # 将特征数据插入到数据库表格中
    feature_data.to_sql('features', conn, if_exists='append', index=False)
    
    # 关闭数据库连接
    conn.close()

# 获取所有的acct_num列表
acct_nums = data['acct_num'].unique()

# 使用多进程并行处理特征工程任务
num_processes = multiprocessing.cpu_count()  # 使用CPU核心数作为进程数
print(num_processes)
num_processes = num_processes-2

with multiprocessing.Pool(num_processes) as pool:
    pool.map(process_acct_num, acct_nums)


8


Process SpawnPoolWorker-3:
Process SpawnPoolWorker-4:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'process_acct_num' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Libra

KeyboardInterrupt: 

In [65]:

import sqlite3
import pandas as pd

# 连接到数据库
conn = sqlite3.connect('feature.db')

# 执行SQL查询并将结果存储在DataFrame中
query = "SELECT * FROM feature"  # 假设表名为'features'
result_df = pd.read_sql_query(query, conn)

# 关闭数据库连接
conn.close()

# 输出查询结果
result_df


,acct_num,txn_amt,txn_amt_variance,time_diff_mean
0,835030,542704.272727,7.246887e+10,10.875000
1,665477,468597.000000,9.419508e+10,-46.869565
2,814594,497460.055556,8.078200e+10,-22.823529
3,602122,443145.960000,7.391189e+10,6.791667
4,336394,478939.576923,8.708509e+10,36.840000
...,...,...,...,...
95,345388,541608.944444,1.125646e+11,3.705882
96,473544,567113.000000,7.576746e+10,50.222222
97,266370,440462.916667,8.494782e+10,49.818182
98,346610,405910.533333,1.035042e+11,-37.642857


In [64]:
data = pd.read_csv('./data/transaction.csv')
data[data['acct_num'] == 665477]

,acct_num,txn_date,txn_time,drcr,txn_amt
5,665477,2023-10-29,17:08,1,325214
88,665477,2023-10-22,04:07,1,393824
278,665477,2023-10-28,16:26,2,900828
533,665477,2023-10-20,03:46,1,122151
537,665477,2023-10-22,05:28,1,237975
624,665477,2023-10-26,01:49,1,84277
711,665477,2023-10-21,15:04,1,42885
792,665477,2023-10-30,02:26,2,711043
1016,665477,2023-09-26,11:21,2,486642
1025,665477,2023-09-25,11:29,2,549376


In [46]:
from feature import calculate_time_diff
data = calculate_time_diff(data, acct_num)

In [47]:
def calculate_average_time_diff(data):
    average_time_diff = data.groupby('acct_num')['time_diff'].mean().reset_index()
    
    return average_time_diff
calculate_average_time_diff(data)


,acct_num,time_diff
0,206953,8.0


In [49]:
data

,acct_num,txn_date,txn_time,time_diff
1651,206953,2023-09-21,1900-01-01 16:43:00,NaN
1972,206953,2023-09-21,1900-01-01 19:28:00,165.0
1396,206953,2023-09-24,1900-01-01 19:07:00,-21.0
1706,206953,2023-09-28,1900-01-01 01:16:00,-1071.0
1999,206953,2023-09-28,1900-01-01 02:02:00,46.0
1639,206953,2023-09-28,1900-01-01 21:26:00,1164.0
1821,206953,2023-09-29,1900-01-01 09:07:00,-739.0
1816,206953,2023-09-30,1900-01-01 04:21:00,-286.0
1560,206953,2023-09-30,1900-01-01 08:20:00,239.0
1740,206953,2023-09-30,1900-01-01 21:29:00,789.0


In [54]:
import sqlite3

# 连接到数据库
conn = sqlite3.connect('feature.db')
cursor = conn.cursor()

# 添加一个名为 "time_diff_mean" 的列到 "feature" 表
# add_column_sql = "ALTER TABLE feature ADD COLUMN time_diff_mean FLOAT"
# cursor.execute(add_column_sql)

add_column_sql = "ALTER TABLE feature ADD COLUMN time_diff TIME"
cursor.execute(add_column_sql)

# 提交更改并关闭连接
conn.commit()
conn.close()
